<a href="https://colab.research.google.com/github/karim-mammadov/Kaggle-Datasets-MyMLProject/blob/main/Bank_Customer_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shantanudhakadd/bank-customer-churn-prediction

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/bank-customer-churn-prediction.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import pandas as pd

# DATA CLEANING and DATA VISUALIZATION

In [ ]:
df = pd.read_csv('/content/Churn_Modelling.csv')
df

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [ ]:
df['BalanceSalaryRatio'] = df['Balance'] / df['EstimatedSalary']

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['Gender'] = le.fit_transform(df['Gender'])
# df['Geography'] = le.fit_transform(df['Geography'])

In [ ]:
import seaborn as sns
sns.countplot(x='Exited', data=df);

In [ ]:
sns.countplot(x='Geography', hue='Exited', data=df);

In [ ]:
sns.boxplot(x='Exited', y='Age', data=df);

In [ ]:
sns.histplot(data=df, x='CreditScore', hue='Exited', kde=True)
sns.histplot(data=df, x='Balance', hue='Exited', kde=True);

In [ ]:
import matplotlib.pyplot as plt
df_corr = df.corr(numeric_only=True)
df_corr

plt.figure(figsize=(7,7))
sns.heatmap(df_corr, annot=True);

# Build a Model

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
X = df.drop('Exited', axis=1)
y = df['Exited'].copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
num_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

In [ ]:
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

transformer = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

full_pipeline_1 = Pipeline(steps=[
    ('transformer', transformer),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [ ]:
full_pipeline_1.fit(X_train,y_train)

In [ ]:
full_pipeline_1.score(X_train, y_train), full_pipeline_1.score(X_test, y_test)

In [ ]:
estimator = DecisionTreeClassifier(random_state=42, min_samples_leaf=2, max_depth=10)

full_pipeline_2 = Pipeline(steps=[
    ('transformer', transformer),
    ('estimator', DecisionTreeClassifier(random_state=42, min_samples_leaf=1, max_depth=8))
])

In [ ]:
full_pipeline_2.fit(X_train,y_train)

In [ ]:
full_pipeline_2.score(X_train, y_train), full_pipeline_2.score(X_test, y_test)

In [ ]:
estimator_2 = RandomForestClassifier(random_state=42, min_samples_leaf=2, max_depth=10)

full_pipeline_3 = Pipeline(steps=[
    ('transformer', transformer),
    ('estimator_2', RandomForestClassifier(random_state=42, min_samples_leaf=1, max_depth=8))
])

In [ ]:
full_pipeline_3.fit(X_train, y_train)

In [ ]:
full_pipeline_3.score(X_train, y_train), full_pipeline_3.score(X_test, y_test)

In [ ]:
estimator_3 = LogisticRegression(max_iter= 1000)

full_pipeline_4 = Pipeline(steps=[
    ('transformer', transformer),
    ('estimator_3', LogisticRegression(max_iter= 1000))
])

In [ ]:
full_pipeline_4.fit(X_train, y_train)

In [ ]:
full_pipeline_4.score(X_train, y_train), full_pipeline_4.score(X_test, y_test)

In [ ]:
estimator_4 = GradientBoostingClassifier(random_state=42)

full_pipeline_5 = Pipeline(steps=[
    ('transformer', transformer),
    ('estimator_4', GradientBoostingClassifier(random_state=42))
])

In [ ]:
full_pipeline_5.fit(X_train, y_train)

In [ ]:
full_pipeline_5.score(X_train, y_train), full_pipeline_5.score(X_test, y_test)